In [2]:

from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import numpy as np


batch_size = 64


train_dataset = datasets.MNIST(root='./data/',
                                train=True,
                                transform=transforms.ToTensor(),
                                download=True)
test_dataset = datasets.MNIST(root='./data/',
                                train=False,
                                transform=transforms.ToTensor())


train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                            batch_size=batch_size,
                                            shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                            batch_size=batch_size,
                                            shuffle=False)


class IEEC(nn.Module):
    

    def __init__(self):
        super(IEEC, self).__init__()
        
        self.conv1_out_np = np.zeros((1, 3, 24, 24))
        self.avp1_out_np = np.zeros((1, 3, 12, 12))
        self.conv2_out_np = np.zeros((1, 3, 8, 8))
        self.avp2_out_np = np.zeros((1, 3, 4, 4))
        self.fc_in_np = np.zeros((1, 48))
        self.fc_out_np = np.zeros((1, 10))
        

        self.conv1 = nn.Conv2d(1, 3, kernel_size=5, bias = False) 
        self.conv2 = nn.Conv2d(3, 3, kernel_size=5, bias = False)
        self.mp = nn.AvgPool2d(2)
        self.fc_1 = nn.Linear(48, 10, bias = False)
        
        
        
    def forward(self, x):
        in_size = x.size(0)
        x = self.conv1(x)
        self.conv1_out_np = x.detach().numpy()
        x = F.relu(self.mp(x))
        self.avp1_out_np = x.detach().numpy()
        x = self.conv2(x)
        self.conv2_out_np = x.detach().numpy()
        x = F.relu(self.mp(x))
        self.avp2_out_np = x.detach().numpy()
        x = x.view(in_size, -1)
        self.fc_in_np = x.detach().numpy()
        x = self.fc_1(x)
        self.fc_out_np = x.detach().numpy()
        
        return F.log_softmax(x)
    
   
model = IEEC()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)


def train(epoch):

    model.train()

    for batch_idx, (data, target) in enumerate(train_loader):
        

        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        
        loss.backward()
        optimizer.step()

        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test():
    model.eval()
    test_loss = 0
    correct = 0
    
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).item()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
          
    test_loss /= len(test_loader.dataset)
    
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
for epoch in range(1, 10):
    train(epoch)
    test()

C:\Users\chanh\AppData\Local\Temp\ipykernel_74180\1879146444.py:69: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.290342
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.288253
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.293523
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.303753
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.282339
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.253622
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.264005
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.237305
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.286587
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.282927
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.226380
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.228175
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.160106
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.159914
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.052126
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.994421
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.746433
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.753162
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.415732
Train Epoch: 1 [12160/60000 (20%)]	Loss: 1.285214
Train Epoch: 1 [12800/60000 (

C:\Users\chanh\AppData\Local\Temp\ipykernel_74180\1879146444.py:102: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True), Variable(target)



Test set: Average loss: 0.3875, Accuracy: 8860/10000 (89%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.262854
Train Epoch: 2 [640/60000 (1%)]	Loss: 0.306302
Train Epoch: 2 [1280/60000 (2%)]	Loss: 0.309740
Train Epoch: 2 [1920/60000 (3%)]	Loss: 0.417463
Train Epoch: 2 [2560/60000 (4%)]	Loss: 0.160288
Train Epoch: 2 [3200/60000 (5%)]	Loss: 0.250967
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.318191
Train Epoch: 2 [4480/60000 (7%)]	Loss: 0.584084
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.368598
Train Epoch: 2 [5760/60000 (10%)]	Loss: 0.382919
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.445712
Train Epoch: 2 [7040/60000 (12%)]	Loss: 0.283825
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.380465
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.524636
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.351120
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.471046
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.370606
Train Epoch: 2 [10880/60000 (18%)]	Loss: 0.359901
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.586549
Train Epoch: 2 [12

In [17]:
int_conv1_weight_1 =  torch.tensor((model.conv1.weight.data[0][0]*128), dtype = torch.int32)
int_conv1_weight_2 =  torch.tensor((model.conv1.weight.data[1][0]*128), dtype = torch.int32)
int_conv1_weight_3 =  torch.tensor((model.conv1.weight.data[2][0]*128), dtype = torch.int32)

print("Signed")
print(int_conv1_weight_1)
print(int_conv1_weight_2)
print(int_conv1_weight_3)

# signed int => unsigned int
for i in range(5):
    for j in range(5):
        if int_conv1_weight_1[i][j] < 0:
            int_conv1_weight_1[i][j] += 256
        if int_conv1_weight_2[i][j] < 0:
            int_conv1_weight_2[i][j] += 256
        if int_conv1_weight_3[i][j] < 0:
            int_conv1_weight_3[i][j] += 256


print ("Unsigned")
print(int_conv1_weight_1)
print(int_conv1_weight_2)
print(int_conv1_weight_3)

np.savetxt('conv1_weight_1.txt', int_conv1_weight_1, fmt='%1.2x',delimiter = " ")
np.savetxt('conv1_weight_2.txt', int_conv1_weight_2, fmt='%1.2x',delimiter = " ")
np.savetxt('conv1_weight_3.txt', int_conv1_weight_3, fmt='%1.2x',delimiter = " ")

Signed
tensor([[ 81,  80,  71,  64,  98],
        [ 36,  58,  32,  64,  81],
        [-21, -35, -18, -16,   2],
        [-83, -55, -58, -62, -30],
        [-53, -55, -50, -27, -22]], dtype=torch.int32)
tensor([[ 45,  43,  38,  42,  35],
        [ -6,   5,   8,  13,   2],
        [-26, -37,  -2, -30,   0],
        [-34, -36, -22,   6,  -3],
        [-17,  15,  -2,  -8,  -2]], dtype=torch.int32)
tensor([[ -6, -27, -12, -16, -31],
        [ 20,  38,  40,  43,  33],
        [ 34,  66,  50,  77,  61],
        [ 67,  84,  92,  65,  57],
        [ 61,  79,  72,  46,  42]], dtype=torch.int32)
Unsigned
tensor([[ 81,  80,  71,  64,  98],
        [ 36,  58,  32,  64,  81],
        [235, 221, 238, 240,   2],
        [173, 201, 198, 194, 226],
        [203, 201, 206, 229, 234]], dtype=torch.int32)
tensor([[ 45,  43,  38,  42,  35],
        [250,   5,   8,  13,   2],
        [230, 219, 254, 226,   0],
        [222, 220, 234,   6, 253],
        [239,  15, 254, 248, 254]], dtype=torch.int32)
tensor([[

C:\Users\chanh\AppData\Local\Temp\ipykernel_15268\4045124262.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  int_conv1_weight_1 =  torch.tensor((model.conv1.weight.data[0][0]*128), dtype = torch.int32)
C:\Users\chanh\AppData\Local\Temp\ipykernel_15268\4045124262.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  int_conv1_weight_2 =  torch.tensor((model.conv1.weight.data[1][0]*128), dtype = torch.int32)
C:\Users\chanh\AppData\Local\Temp\ipykernel_15268\4045124262.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  int_conv1_weight_3 

In [4]:
############## Conv2 가중치 값 HEX 추출 ############

print(np.shape(model.conv2.weight))

# float => int
int_conv2_weight_11 =  torch.tensor((model.conv2.weight.data[0][0]* 128), dtype = torch.int32)
int_conv2_weight_12 =  torch.tensor((model.conv2.weight.data[0][1]* 128), dtype = torch.int32)
int_conv2_weight_13 =  torch.tensor((model.conv2.weight.data[0][2]* 128), dtype = torch.int32)

int_conv2_weight_21 =  torch.tensor((model.conv2.weight.data[1][0] * 128), dtype = torch.int32)
int_conv2_weight_22 =  torch.tensor((model.conv2.weight.data[1][1] * 128), dtype = torch.int32)
int_conv2_weight_23 =  torch.tensor((model.conv2.weight.data[1][2] * 128), dtype = torch.int32)

int_conv2_weight_31 =  torch.tensor((model.conv2.weight.data[2][0] * 128), dtype = torch.int32)
int_conv2_weight_32 =  torch.tensor((model.conv2.weight.data[2][1] * 128), dtype = torch.int32)
int_conv2_weight_33 =  torch.tensor((model.conv2.weight.data[2][2] * 128), dtype = torch.int32)

print ("Signed")
print(int_conv2_weight_11)
print(int_conv2_weight_12)
print(int_conv2_weight_13, '\n')

print(int_conv2_weight_21)
print(int_conv2_weight_22)
print(int_conv2_weight_23, '\n')

print(int_conv2_weight_31)
print(int_conv2_weight_32)
print(int_conv2_weight_33, '\n')

# signed int => unsigned int
for i in range(5):
    for j in range(5):
        if int_conv2_weight_11[i][j] < 0:
            int_conv2_weight_11[i][j] += 256
        if int_conv2_weight_12[i][j] < 0:
            int_conv2_weight_12[i][j] += 256
        if int_conv2_weight_13[i][j] < 0:
            int_conv2_weight_13[i][j] += 256
            
        if int_conv2_weight_21[i][j] < 0:
            int_conv2_weight_21[i][j] += 256
        if int_conv2_weight_22[i][j] < 0:
            int_conv2_weight_22[i][j] += 256
        if int_conv2_weight_23[i][j] < 0:
            int_conv2_weight_23[i][j] += 256
            
        if int_conv2_weight_31[i][j] < 0:
            int_conv2_weight_31[i][j] += 256
        if int_conv2_weight_32[i][j] < 0:
            int_conv2_weight_32[i][j] += 256
        if int_conv2_weight_33[i][j] < 0:
            int_conv2_weight_33[i][j] += 256

print ("Unsigned")
print(int_conv2_weight_11)
print(int_conv2_weight_12)
print(int_conv2_weight_13, '\n')

print(int_conv2_weight_21)
print(int_conv2_weight_22)
print(int_conv2_weight_23, '\n')

print(int_conv2_weight_31)
print(int_conv2_weight_32)
print(int_conv2_weight_33, '\n')

np.savetxt('conv2_weight_11.txt', int_conv2_weight_11, fmt='%1.2x',delimiter = " ")
np.savetxt('conv2_weight_12.txt', int_conv2_weight_12, fmt='%1.2x',delimiter = " ")
np.savetxt('conv2_weight_13.txt', int_conv2_weight_13, fmt='%1.2x',delimiter = " ")

np.savetxt('conv2_weight_21.txt', int_conv2_weight_21, fmt='%1.2x',delimiter = " ")
np.savetxt('conv2_weight_22.txt', int_conv2_weight_22, fmt='%1.2x',delimiter = " ")
np.savetxt('conv2_weight_23.txt', int_conv2_weight_23, fmt='%1.2x',delimiter = " ")

np.savetxt('conv2_weight_31.txt', int_conv2_weight_31, fmt='%1.2x',delimiter = " ")
np.savetxt('conv2_weight_32.txt', int_conv2_weight_32, fmt='%1.2x',delimiter = " ")
np.savetxt('conv2_weight_33.txt', int_conv2_weight_33, fmt='%1.2x',delimiter = " ")


torch.Size([3, 3, 5, 5])
Signed
tensor([[  8,  18,   6, -15, -38],
        [ 13,  20,  37,  32,  22],
        [ 52,  38,  65,  72,  40],
        [ 53,  48,  67,  53,  15],
        [ 25,  23,  10,  -9, -11]], dtype=torch.int32)
tensor([[  0, -10,  13,  28, -16],
        [ 19,   6,  10,  18,  19],
        [ -9,  20,  20,  24,  25],
        [ 10,   6,  14,  29,   9],
        [  3,  -3,  17,   0,   0]], dtype=torch.int32)
tensor([[ 29, -11, -11,  67,  29],
        [ -5,  -6,   7,  70,  30],
        [-25, -14, -15, -14,  14],
        [ -9, -26, -60, -40, -24],
        [ 43,  36,   4,  29,   6]], dtype=torch.int32) 

tensor([[ 13,   0,  14,   0,  -1],
        [  5,   0, -22,  -6, -12],
        [-23, -21, -42, -33, -13],
        [-25, -49, -49, -22,  -1],
        [-31, -34, -47, -27,  -7]], dtype=torch.int32)
tensor([[ -1,   2,  -1,   3,  10],
        [  8,   0, -11, -13,   1],
        [  9,   0, -15,  -2,  11],
        [ -3, -13, -27, -16, -18],
        [-11, -16,   4,  -7,  -5]], dtype=torc

C:\Users\chanh\AppData\Local\Temp\ipykernel_15268\469399959.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  int_conv2_weight_11 =  torch.tensor((model.conv2.weight.data[0][0]* 128), dtype = torch.int32)
C:\Users\chanh\AppData\Local\Temp\ipykernel_15268\469399959.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  int_conv2_weight_12 =  torch.tensor((model.conv2.weight.data[0][1]* 128), dtype = torch.int32)
C:\Users\chanh\AppData\Local\Temp\ipykernel_15268\469399959.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  int_conv2_weight_1

In [5]:
print(np.shape(model.fc_1.weight))
print((model.fc_1.weight * 128).int())

int_fc_weight = (model.fc_1.weight * 128).int()

# signed int => unsigned int
for i in range(10):
    for j in range(48):
        if int_fc_weight[i][j] < 0 :
            int_fc_weight[i][j] += 256
        
print(int_fc_weight)

np.savetxt('fc_weight.txt', int_fc_weight, fmt='%1.2x',delimiter = " ")

torch.Size([10, 48])
tensor([[  1,  12,  27,   1,  -1,  -3,  -5, -10,  -3, -64, -26,  27,  30, -36,
          24,  43,  12,   6,   2,  -8,  34,  34, -13,   5,  28,  11,  12,  -4,
          -7,  -5,  13,   9,   8, -13,   6,  14, -11, -16, -30, -36, -40,  -8,
         -25, -41,   9,  34,  16,  -9],
        [-36,   0, -69,   4, -15,  28,  -3, -16, -56,  48, -39,   7, -31,  59,
         -50,  10, -13, -23,  44, -16, -20,  -2,  10, -52, -26,  33, -21,  22,
          45, -13, -24,   2, -38,   4, -21, -22,  45, -11, -13,  34,  15, -35,
          -5,  27,  -5,  30,  24,  38],
        [ 25,  38,  19, -44,  30, -13, -13, -33, -18, -40, -18,   0,   0, -13,
          38,  37, -59, -12, -42,  -1,  13,  10,  -2,  27,  65,  31, -16, -29,
          14,   9,   5,  19,  33,  10,  11,  -7,  47,  17,  -7,   7,  17, -10,
           8,   4,  20, -18,  37,  74],
        [ 30,  35,  21, -38,   2,   8,  -7,  21,  17,  26,   9, -16,  22,   2,
          29,  22, -58, -48,   4,  16, -21, -12, -12, -24,  -3, -29, 